<a href="https://colab.research.google.com/github/41371122h-lichi/lichi_thursday/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80HW1_%E5%87%BA%E9%81%8A%E6%94%AF%E5%87%BA%E9%80%9F%E7%AE%97%E8%88%87%E5%88%86%E6%94%A4_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **出遊支出速算與分攤（作業一）**

跟朋友出去玩不知道怎麼分錢嗎？

想要有個一目瞭然的記帳方式嗎？

本程式可以填寫日期、時間、品項、數量、總金額、支付人

還可以幫忙計算目前朋友欠你的錢，跟你欠朋友的錢喔！

**因為會自動分錢，僅限和朋友出遊時記帳用**

*   日期(格式YYYY-MM-DD)
*   品項
*   數量
*   總金額
*   支付人
*   欠款人數(購買總人數-1)

項目太多分不清楚時，可以按下篩選器，更方便大家計算！

出去玩時跟朋友一起來記帳吧！

GoogleSheet:https://docs.google.com/spreadsheets/d/1E4PRsZXUN7zRudfHCW_ZA9BIK_Ho_dskIPGlHXHKDV8/edit?usp=sharing

In [94]:
import gspread
import pandas as pd
import datetime
import gradio as gr
from google.colab import auth
from google.auth import default

In [95]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1E4PRsZXUN7zRudfHCW_ZA9BIK_Ho_dskIPGlHXHKDV8/edit?usp=sharing"

REQUIRED_COLUMNS = ["日期","分類","品項","數量","總金額","自付金額","支付方式",
                    "支付人","是否自己付","欠款人數","被欠款金額","個人欠款金額"]

# 定義品項清單，你可以自行增加或修改
CATEGORY_CHOICES = ["餐食", "交通", "住宿", "門票", "其他"]

_gc, _ws = None, None
_total_people = 1 # 預設人數
_worksheet_name = "新工作表"

In [ ]:
def _ensure_auth_and_ws(sheet_name=None):
    global _gc, _ws
    auth.authenticate_user()
    creds, _ = default()
    _gc = gspread.authorize(creds)
    gs = _gc.open_by_url(SHEET_URL)
    if sheet_name:
        try:
            _ws = gs.worksheet(sheet_name)
        except gspread.WorksheetNotFound:
            _ws = gs.add_worksheet(title=sheet_name, rows="100", cols="20")
            _ws.append_row(REQUIRED_COLUMNS, value_input_option="USER_ENTERED")
    return _ws

def set_total_people(n):
    global _total_people
    try:
        n_int = int(n)
        if n_int < 1 or n_int > 30:
            return "❌ 人數需介於 1~30 人"
        _total_people = n_int
        return f"✅ 已設定這次共有 {_total_people} 人出遊，請按旁邊的**新增支出**開始記帳"
    except:
        return "❌ 請輸入有效的人數"

def update_owe_people_max(n):
    """
    根據總人數更新欠款人數的上限。
    """
    try:
        n_int = int(n)
        if n_int > 0:
            return gr.update(maximum=n_int - 1)
        else:
            return gr.update(maximum=0)
    except:
        return gr.update()

def _read_df():
    ws = _ws
    values = ws.get_all_values()
    if not values:
        return pd.DataFrame(columns=REQUIRED_COLUMNS)
    df = pd.DataFrame(values[1:], columns=values[0])
    for col in ["總金額","自付金額","欠款人數","被欠款金額","個人欠款金額"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0.0)
    return df

def setup_sheet_and_people(sheet_name, people_n):
    """
    建立工作表並設定人數，並清除輸入欄位。
    """
    _ensure_auth_and_ws(sheet_name)
    msg = set_total_people(people_n)
    success_msg = f"✅ 工作表「{sheet_name}」建立完成\n{msg}"
    return success_msg, gr.update(value=""), gr.update(value=1)

# ------------------- 記帳 -------------------
def add_expense(date_str, category, item, count, amount, payer, is_self_paid, owe_people, is_last_item):
    try:
        if not date_str:
            date_str = datetime.date.today().strftime("%Y-%m-%d")

        category = (category or "未填").strip()
        item = (item or "未填").strip()
        count = (count or "未填").strip()
        payer = (payer or "匿名").strip()

        try:
            amount_val = float(amount)
        except:
            return "❌ 總金額格式錯誤，請輸入數字", None, None, None, \
                   gr.update(value=datetime.date.today().strftime("%Y-%m-%d")), \
                   gr.update(value=""), gr.update(value=""), gr.update(value=""), \
                   gr.update(value=""), gr.update(value="否"), gr.update(value=0), gr.update(value=False)

        if is_self_paid == "是":
            total_payers = int(owe_people) + 1
            per_person_cost = round(amount_val / total_payers, 2)
            self_amount = per_person_cost
            owe_people_val = int(owe_people)
            owe_amount_val = round(amount_val - self_amount, 2)
            owe_amount_user = 0.0
        else:
            self_amount = 0.0
            owe_people_val = int(owe_people) if owe_people > 0 else 1
            owe_amount_user = round(amount_val / (owe_people_val + 1), 2)
            owe_amount_val = 0.0

        ws = _ws
        ws.append_row([date_str, category, item, count, amount_val, self_amount,
                        "現金", payer, is_self_paid, owe_people_val, owe_amount_val, owe_amount_user],
                        value_input_option="USER_ENTERED")

        df = _read_df()
        total = df["總金額"].sum().round(2) if not df.empty else 0.0
        by_cat = df.groupby("分類")["總金額"].sum().reset_index()

        if is_last_item:
            msg = "✅ 資料已新增。\n\n**本行程記帳結束！**\n\n若要新增下一個行程，請回到 **工作表&人數設定** 頁面。"
        else:
            msg = f"✅ 已新增：{date_str} ｜{category}｜{item}｜總金額 {amount_val}｜自付 {self_amount}｜是否自己付 {is_self_paid}｜欠款人數 {owe_people_val}｜被欠款金額 {owe_amount_val}｜個人欠款金額 {owe_amount_user}"

        return (
            msg, total, by_cat, df,
            gr.update(value=datetime.date.today().strftime("%Y-%m-%d")),
            gr.update(value=""),
            gr.update(value=""),
            gr.update(value=""),
            gr.update(value=""),
            gr.update(value=""),
            gr.update(value="否"),
            gr.update(value=0),
            gr.update(value=False)
        )
    except Exception as e:
        return f"❌ 新增失敗：{e}", None, None, None, \
               gr.update(value=datetime.date.today().strftime("%Y-%m-%d")), \
               gr.update(value=None), gr.update(value=None), gr.update(value=None), \
               gr.update(value=None), gr.update(value="否"), gr.update(value=0), gr.update(value=False)

# Gradio UI 介面
with gr.Blocks(title="出遊支出速算與分攤") as demo:
    gr.Markdown("## 🧾 出遊支出速算與分攤")

    with gr.Tabs():
        with gr.Tab("工作表 & 人數設定"):
            ws_name_in = gr.Textbox(label="自訂工作表名稱")
            people_in = gr.Number(label="這次共有幾個人出遊（1~30）", value=1, minimum=1)
            setup_btn = gr.Button("建立工作表")
            setup_msg = gr.Markdown()
            setup_btn.click(fn=setup_sheet_and_people, inputs=[ws_name_in, people_in], outputs=[setup_msg, ws_name_in, people_in])

        with gr.Tab("新增支出"):
            gr.Markdown("### 請填寫支出明細")

            with gr.Row():
                date_in = gr.Textbox(label="日期 YYYY-MM-DD", value=datetime.date.today().strftime("%Y-%m-%d"))
                cat_in = gr.Dropdown(choices=CATEGORY_CHOICES, label="分類")
                item_in = gr.Textbox(label="品項", placeholder="如：麵、車錢")

            with gr.Row():
                cou_in = gr.Textbox(label="數量", placeholder="無須單位")
                amt_in = gr.Textbox(label="總金額", placeholder="無須單位")

            with gr.Row():
                payer_in = gr.Textbox(label="支付人", placeholder="若無填寫則顯示匿名")
                is_self_paid = gr.Radio(choices=["否","是"], value="否", label="是否自己付")
                owe_people_in = gr.Number(label="欠款人數", value=0, minimum=0)

            is_last_item = gr.Checkbox(label="是否為此行程的最後一筆資料？")
            add_btn = gr.Button("新增到工作表")

            gr.Markdown("---")
            add_msg = gr.Markdown()
            gr.Markdown("### 統計數據")
            total_out = gr.Number(label="目前總額", interactive=False)
            cat_df = gr.Dataframe(label="分類小計", interactive=False)
            raw_preview = gr.Dataframe(label="（預覽）最近資料", interactive=False)

            people_in.change(fn=update_owe_people_max, inputs=people_in, outputs=owe_people_in)

            add_btn.click(
                fn=add_expense,
                inputs=[date_in, cat_in, item_in, cou_in, amt_in, payer_in,
                        is_self_paid, owe_people_in, is_last_item],
                outputs=[add_msg, total_out, cat_df, raw_preview,
                         date_in, cat_in, item_in, cou_in, amt_in, payer_in,
                         is_self_paid, owe_people_in, is_last_item]
            )

demo.launch()